In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("./adult.data.csv")
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
#Сколько мужчин и женщин (признак sex) представлено в этом наборе данных?
data['sex'].value_counts()


Male      21790
Female    10771
Name: sex, dtype: int64

In [4]:
#2. Каков средний возраст (признак age) женщин?
data[data['sex']=='Female']['age'].mean()

36.85823043357163

In [5]:
#Какова доля граждан Германии (признак native-country)?
data[data['native-country'] =='Germany'].count() / data.count()

age               0.004207
workclass         0.004207
fnlwgt            0.004207
education         0.004207
education-num     0.004207
marital-status    0.004207
occupation        0.004207
relationship      0.004207
race              0.004207
sex               0.004207
capital-gain      0.004207
capital-loss      0.004207
hours-per-week    0.004207
native-country    0.004207
salary            0.004207
dtype: float64

In [6]:
#4-5. Каковы средние значения и среднеквадратичные отклонения возраста тех, кто получает более 50K в год (признак salary) и тех, кто получает менее 50K в год?
# Средний возраст с зп больше 50к
data['age'][data['salary'] == '>50K'].mean()

44.24984058155847

In [7]:
# Средний возраст с зп меньше 50к
data['age'][data['salary'] == '<=50K'].mean()

36.78373786407767

In [8]:
#Стандартное откл возраста с зп > 50к
np.std(data['age'][data['salary'] == '>50K'], axis = 0)

10.518356927661575

In [9]:
#Стандартное откл возраста с зп < 50r
np.std(data['age'][data['salary'] == '<=50K'], axis = 0)

14.019804910115214

In [10]:
#Правда ли, что люди, которые получают больше 50k, имеют как минимум высшее образование? (признак education – Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters или Doctorate)
ed1 = data[(data['salary'] == '>50K') & (data['education'] == 'Bachelors')].count()
ed2 = data[(data['salary'] == '>50K') & (data['education'] == 'Prof-school')].count()
ed3 = data[(data['salary'] == '>50K') & (data['education'] == 'Assoc-acdm')].count()
ed4 = data[(data['salary'] == '>50K') & (data['education'] == 'Assoc-voc')].count()
ed5 = data[(data['salary'] == '>50K') & (data['education'] == 'Masters')].count()
ed6 = data[(data['salary'] == '>50K') & (data['education'] == 'Doctorate')].count()
ed = ed1['salary' ] + ed2['salary'] + ed3['salary'] + ed4['salary'] + ed5['salary'] + ed6['salary']
data['salary' ].count() - ed == 0

False

In [11]:
# Через множество выглядит лучше 
set(data[data['salary'] == '>50K']['education']) == {
 'Assoc-acdm',
 'Assoc-voc',
 'Bachelors',
 'Doctorate',
 'Masters',
 'Prof-school'}

False

In [12]:
# Выведите статистику возраста для каждой расы (признак race) и каждого пола. 
data.groupby(['race', 'sex'])[['age']].agg([np.min, np.max, np.mean, np.std,])

age                           
                          amin amax       mean        std
race               sex                                   
Amer-Indian-Eskimo Female   17   80  37.117647  13.114991
                   Male     17   82  37.208333  12.049563
Asian-Pac-Islander Female   17   75  35.089595  12.300845
                   Male     18   90  39.073593  12.883944
Black              Female   17   90  37.854019  12.637197
                   Male     17   90  37.682600  12.882612
Other              Female   17   74  31.678899  11.631599
                   Male     17   77  34.654321  11.355531
White              Female   17   90  36.811618  14.329093
                   Male     17   90  39.652498  13.436029

In [13]:
#Среди кого больше доля зарабатывающих много (>50K) это неправильно
data_male = data[(data['sex'] == 'Male') & (data['salary'] == '>50K')]

pd.crosstab(data_male['salary'], data_male['marital-status'], normalize=True) * 100 

# Себе на будующее, это неправильно, это соотношении богатых по признаку.

marital-status,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed
salary,,,,,,,
>50K,4.262984,0.060042,89.132393,0.345242,4.878415,0.735515,0.58541


In [87]:
#Среди кого больше доля зарабатывающих много (>50K) Правильное решение
#data_male[1] - заработок >50к
#data_male[0] - заработок <=50к
marrieds = [0, 0]  #женатые [заработок >50к, заработок <50к]  
no_marrieds = [0, 0]  #неженатые [заработок >50к, заработок <50к] 

marital_status = set(data[data['salary'] == '>50K'] ['marital-status'])          
for status in marital_status:
    data_male = data[(data['sex'] == 'Male') & (data['marital-status'] == status)]['salary'].value_counts()
    if (status[0:1] == 'M'):
        marrieds[0] = marrieds[0] + data_male[0]
        marrieds[1] = marrieds[1] + data_male[1]        
        #print(data_male[0], data_male[1], marrieds[1])       
    else:
        no_marrieds[0] = no_marrieds[0] + data_male[0]
        no_marrieds[1] = no_marrieds[1] + data_male[1]
        
print(no_marrieds, marrieds)     

print('Доля женатых с з/п больше 50к', marrieds[1] / np.sum(marrieds))
print('Доля неженатых с з/п больше 50к', no_marrieds[1] / np.sum(no_marrieds))



[7552, 697] [7576, 5965]
Доля женатых с з/п больше 50к 0.4405139945351156
Доля неженатых с з/п больше 50к 0.08449509031397745
